# CityBikes

##### Send a request to CityBikes for the city of your choice. 

For getting data from Citybikes API, I created an account in [CityBikes](https://citybik.es/).
Then, for the city of Toronto, ON information about bike stations was retrieved.
Following parameters considered for processing:

    a) uid        
    b) name        
    c) longitude
    d) latitude   
    e) free_bikes  
    e) empty_slots

In [2]:
# This code send request to get 'Toronto, ON' CityBikes information 

import requests
from IPython.display import JSON #for nice JSON (or dict) display in JupyterLab (not currently supported in Jupyter Notebook)
import json
from pandas import json_normalize

#Get data for certain network_id
network_id = 'bixi-toronto' # for 'Toronto, ON'
network = requests.get("http://api.citybik.es/v2/networks/"+network_id).json()

# All Stations Informations
network_json=network['network'].get('stations')

# Convert to Dataframe 
df=json_normalize(network_json)
df.head()


,empty_slots,free_bikes,id,latitude,longitude,name,timestamp,extra.address,extra.altitude,extra.ebikes,extra.has_ebikes,extra.last_updated,extra.normal_bikes,extra.payment,extra.payment-terminal,extra.renting,extra.returning,extra.slots,extra.uid,extra.post_code
0,4,14,fb337bbed72e2be090071e199899b2be,43.665269,-79.319796,Queen St E / Woodward Ave,2023-02-27T03:23:09.554000Z,Queen St E / Woodward Ave,0.0,0,True,1677467972,14,"[key, transitcard, creditcard, phone]",True,1,1,19,7303,NaN
1,1,14,4ff88d5880e71aa40d34cfe5d09b0ca7,43.671420,-79.445947,Primrose Ave / Davenport Rd,2023-02-27T03:23:09.555000Z,Primrose Ave / Davenport Rd,0.0,0,True,1677468107,14,"[key, transitcard, creditcard, phone]",True,1,1,15,7301,NaN
2,19,4,a09c67c0b419654d907c9134b108e328,43.666224,-79.317693,Queen St. E / Rhodes Ave.,2023-02-27T03:23:09.556000Z,Queen St. E / Rhodes Ave.,0.0,0,True,1677468187,4,"[key, transitcard, creditcard, phone]",True,1,1,23,7309,NaN
3,13,12,d6a9daee68070a8b106cfb598d81308c,43.653236,-79.376716,Bond St / Queen St E,2023-02-27T03:23:09.562000Z,Bond St / Queen St E,0.0,0,True,1677467947,12,"[key, transitcard, creditcard, phone]",True,1,1,25,7045,NaN
4,7,8,8f8af40d9388c8a3962559e8681d3db7,43.663722,-79.380288,Church St / Alexander St,2023-02-27T03:23:09.563000Z,Church St / Alexander St,0.0,0,True,1677468042,8,"[key, transitcard, creditcard, phone]",True,1,1,15,7044,NaN


##### Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes). 

In this section, applicable fields (including uid, name, longitude, latitude, free_bikes, and empty_slots) are kept and other fields were deleted.


In [3]:
# Keep extra.uid as uid
df['uid'] = df['extra.uid']

# Delete columns listed in 'list_delete'
list_delete=['id','timestamp','extra.address','extra.altitude','extra.ebikes','extra.has_ebikes','extra.last_updated','extra.normal_bikes','extra.payment','extra.payment-terminal','extra.renting','extra.returning','extra.slots','extra.uid','extra.post_code']

for i in list_delete:
    df = df.drop(i, axis=1)

# Reorder columns
df = df.reindex(columns=['uid','name','longitude','latitude','free_bikes','empty_slots'])
df.head()

,uid,name,longitude,latitude,free_bikes,empty_slots
0,7303,Queen St E / Woodward Ave,-79.319796,43.665269,14,4
1,7301,Primrose Ave / Davenport Rd,-79.445947,43.671420,14,1
2,7309,Queen St. E / Rhodes Ave.,-79.317693,43.666224,4,19
3,7045,Bond St / Queen St E,-79.376716,43.653236,12,13
4,7044,Church St / Alexander St,-79.380288,43.663722,8,7


In [4]:
# Null value detecting
df.isna().mean()

uid            0.0
name           0.0
longitude      0.0
latitude       0.0
free_bikes     0.0
empty_slots    0.0
dtype: float64

In [5]:
# Check for duplicates
print(df.shape)
df[['uid','name']].nunique() 

(653, 6)


uid     653
name    653
dtype: int64

##### Put your parsed results into a DataFrame.

In [3]:
# Save as CSV file
df.to_csv('data/city_bikes.csv', sep=',', encoding='utf-8',index=False)
